# SVM_NN_CNN_Lithological Mapping Using Satellite Data (ASTER, Landsat, Sentinel)

In [ ]:
# Some important and mostly-used packages are already installed on Google-Colab or
# You can run the on Jupiter or Spyder environment after installing required packages
import numpy as np
from osgeo import gdal
import matplotlib.pyplot as plt

In [ ]:
# Some other packages require installation.
# They can be accessed from Github; or installed using "pip".

# After successful installation, import the package.
import rasterio as rio

In [ ]:
# Some packages are very large and you don't need to import the entire library.
# Just work with those methods that you'll work through your project.
from sklearn import svm
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import numpy as np

In [ ]:
# Use rasterio package to open images.
# Data can be read from Google Drive directory.


# For LANDSAT
LANDSAT = rio.open(r"D:\KULIAH\Makan Bang\Argopuro\10. ML\Code\CNN\Dataset\Virtual Raster.tif")
LANDSAT_array = LANDSAT.read()
                   
GT = rio.open(r"D:\KULIAH\Makan Bang\Argopuro\9.5. Data\Data 1.0\Geologi\Peta Geologi Daerah Penelitian\Polygon\Raster Peta Daerah Peneltian.tif")
GT_array = GT.read()


In [ ]:
LANDSAT_array.shape
GT.shape

In [ ]:
# Let's see what these datasets look like.
print(LANDSAT_array.shape)
#print(SENTINEL_array.shape)
print(GT_array.shape)

In [3]:
nRows = LANDSAT_array.shape[1]
nCols = LANDSAT_array.shape[2]
Bands = LANDSAT_array.shape[0]

print(Bands,nRows,nCols)

NameError: name 'LANDSAT_array' is not defined

In [ ]:
Bands

## Data Visualization

Visualization is the best way to become familiar with your dataset. Also, you can summarize the data by looking at some statistical features of the data.



In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(LANDSAT_array[2, :, :], cmap='gray')
plt.show()

In [ ]:
# Visualizing DSM
plt.figure(figsize=(9, 9))
plt.imshow(LANDSAT_array[1, :, :], cmap='jet')
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
# Let's look at it's histogram!
plt.figure(figsize=(10, 7))
plt.grid(axis='y', alpha=0.75)
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('LANDSAT Histogram')
n, bins, patches = plt.hist(x=LANDSAT_array.flatten(), bins=100, color='#0504aa',
                            alpha=0.7, rwidth=0.85)

In [ ]:
#Deal with outliers
#LANDSAT_array[LANDSAT_array > 60] = np.nan

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(LANDSAT_array[0, :, :], cmap='jet')
plt.show()

In [ ]:
# Correct for incorrect values (outliers).
#LANDSAT_array[LANDSAT_array > 50] = np.nan

#plt.figure(figsize=(10, 7))
#plt.grid(axis='y', alpha=0.75)
#plt.xlabel('Value')
#plt.ylabel('Frequency')
#plt.title('LANDSAT Histogram')
#n, bins, patches = plt.hist(x=LANDSAT_array.flatten(), bins=100, color='r',alpha=0.7, rwidth=0.85)

In [ ]:
# Again, visualize the corrected data.
plt.figure(figsize=(10, 7))
plt.imshow(LANDSAT_array[0, :, :], cmap='jet')
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
# OK. What's next? Ground truth data
fig = plt.figure(figsize=(13, 7))
gs = fig.add_gridspec(1, 5)
fig.add_subplot(gs[0, :3]), plt.imshow(GT_array[0, :, :], cmap='gist_ncar')
fig.add_subplot(gs[0, -2:]), plt.imshow(GT_array[0, 200:300, 100:200], cmap='gist_ncar')
plt.show()

In [ ]:
list(np.unique(GT_array))


In [4]:
# How many classes?
GT_array[GT_array == 255] = 0
classes = np.unique(GT_array)
print(classes)

NameError: name 'GT_array' is not defined

In [ ]:
#c = 2
#cls = np.zeros((nRows, nCols), dtype=int)
#cls[GT_array[0, :, :] == c] = 1

c = 2
cls = np.zeros((nRows, nCols), dtype=int)
cls[GT_array[0, :, :] == c] = 1

plt.figure()
plt.imshow(cls)
plt.show()

In [ ]:
# Take a look at individual classes.
plt.figure(figsize=(10, 10))
plt.tight_layout()
for i in range(len(classes)):
  if i < 1: 
    continue
  C = np.zeros((nRows, nCols))
  C[GT_array[0, :, :] == classes[i]] = 1
  plt.subplot(3, 5, i)
  plt.title(str(classes[i]))
  plt.xticks([])
  plt.yticks([])
  plt.imshow(C)
plt.show()

## Train/Test Split

In order to have completely different and unique train/test samples, we divide our ground truth data into two sets of train samples and test samples.

> You can also use [`train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function in scikit-learn.

In [ ]:
GT_array = GT_array[0, :, :].astype(int)
classes = np.unique(GT_array)
print(classes)

In [ ]:
print(GT_array.shape)
print(GT_array.flatten().shape)
print(nRows * nCols)
print(GT_array.flatten()[1000])

In [ ]:
for i in range(20):
  if i in classes:
    print(len(np.where(GT_array.flatten() == i)[0]))

# print(len(np.where(GT_array.flatten() == 9)[0]))
# print(len(np.where(GT_array.flatten() == 2)[0]))
# print(len(np.where(GT_array.flatten() == 4)[0]))

In [ ]:
def split_roi(gt_data, percent):
  import random

  Train = np.zeros_like(gt_data, dtype=int)
  Test = np.zeros_like(gt_data, dtype=int)
  
  labels = list(np.unique(gt_data))
  if 0 in labels: labels.remove(0) 

  for l in labels:
    ind = list(np.where(gt_data.flatten() == l)[0])

    random.shuffle(ind)
    train_inds = ind[:int(percent * len(ind))]
    test_inds = ind[int(percent * len(ind)):]

    temp = np.zeros((nRows*nCols, 1), dtype=int)
    temp[train_inds] = l
    temp = temp.reshape((nRows, nCols))
    Train = Train + temp
    
    temp = np.zeros((nRows*nCols, 1), dtype=int)
    temp[test_inds] = l
    temp = temp.reshape((nRows, nCols))
    Test = Test + temp

  return Train, Test

In [ ]:
Train, Test = split_roi(GT_array, 0.7)

In [ ]:
plt.figure(figsize=(10, 10))
plt.subplot(121), plt.imshow(Train[400:500, 100:200], cmap='jet')
plt.subplot(122), plt.imshow(Test[400:500, 100:200], cmap='jet')
plt.show()

In [ ]:
# Just show a few bands
plt.figure(figsize=(13, 13))
plt.subplot(321), plt.imshow(LANDSAT_array[0, :, :], cmap='gray')
plt.subplot(322), plt.imshow(LANDSAT_array[1, :, :], cmap='gray')
plt.subplot(323), plt.imshow(LANDSAT_array[2, :, :], cmap='gray')
plt.subplot(324), plt.imshow(LANDSAT_array[3, :, :], cmap='gray')
plt.subplot(325), plt.imshow(LANDSAT_array[4, :, :], cmap='gray')
plt.subplot(326), plt.imshow(LANDSAT_array[5, :, :], cmap='gray')
plt.subplot(326), plt.imshow(LANDSAT_array[6, :, :], cmap='gray')
plt.show()

In [ ]:
# Also, we can see the spectral curve of a pixel.
plt.plot(LANDSAT_array[:, 200, 300])

In [ ]:
np.mean(LANDSAT_array[:, Train == c][:, :10], axis=1).shape
x = np.mean(LANDSAT_array[:, Train == c][:, :10], axis=1).shape
print(x)

In [ ]:
# Let's try it for a class.
classes = [1, 2, 3, 4, 5, 6, 7]
plt.figure(figsize=(25, 25))
for c in classes:
  plt.subplot(7, 2, classes.index(c)+1), plt.plot(LANDSAT_array[:, Train == c][:, :10], 'y')
  plt.plot(np.mean(LANDSAT_array[:, Train == c][:, :10], axis=1), 'k')
  plt.ylim([0, 400])
plt.show()

## Data Standardization

[Standardization](https://scikit-learn.org/stable/modules/preprocessing.html#standardization-or-mean-removal-and-variance-scaling) of datasets is to make data look like Gaussian with **zero mean and unit variance**.

In practice we often ignore the shape of the distribution and just transform the data to center it by removing the mean value of each feature, then scale it by dividing non-constant features by their standard deviation.

> We are actually standardizing our datasets.

![picture](https://scikit-learn.org/stable/_images/sphx_glr_plot_all_scaling_002.png)

[Normalization](https://scikit-learn.org/stable/modules/preprocessing.html#normalization) is the process of scaling individual samples to have **unit norm**.




In [ ]:
LANDSAT_array.reshape(7, nRows * nCols).T.shape
# [ # observations (# samples / # pixels),  # features]

In [ ]:
# Prepare data for machine learning operation.
LANDSAT_data = LANDSAT_array.reshape(7, nRows * nCols).T
#SENTINEL_data = SENTINEL_array.reshape(13, nRows * nCols).T

Train_data = Train.reshape(1, nRows * nCols).T
Test_data = Test.reshape(1, nRows * nCols).T

print(LANDSAT_data.shape, Train_data.shape, Test_data.shape)

In [ ]:
# Standardizing data
scaler = StandardScaler()
LANDSAT_standard = scaler.fit(LANDSAT_data)
print(scaler.mean_, scaler.var_)
LANDSAT_standard = LANDSAT_standard.transform(LANDSAT_data)
scaler = StandardScaler()

In [ ]:
# You can do the same operation mannually.
# ASTER_standard = np.zeros_like(ASTER_data, dtype=np.float)
# for i in range(48):
#   ASTER_standard[:, i] = (ASTER_data[:, i] - np.mean(ASTER_data[:, i])) / np.std(ASTER_data[:, i])

In [ ]:
# See how standardization works.
plt.figure(figsize=(10, 10))
for c in classes:
  plt.subplot(7, 2, classes.index(c)+1), plt.plot(LANDSAT_standard[Train.flatten() == c, :][:10, :].T)
plt.show()

# Classification-CNN

In [ ]:
# !pip install pyrsgis
import numpy as np
import random
# import sklearn
from pyrsgis import raster
from pyrsgis.ml import imageChipsFromArray

In [ ]:
# !pip install rasterio

# After successful installation, import the package.
import rasterio as rio

In [ ]:
#ASTER = rio.open('E:/My_Flash64/SVM_NN/ASTER_CNN/Input/AST15.tif')
#ASTER_array = ASTER.read()

#GT = rio.open('E:/My_Flash64/SVM_NN/ASTER_CNN/Input/GT15.tif')
#GT_array = GT.read()

In [ ]:
# Defining file names
featureFile = r"D:\KULIAH\Makan Bang\Argopuro\10. ML\Code\CNN\Dataset\Virtual Raster.tif"
labelFile = r"D:\KULIAH\Makan Bang\Argopuro\9.5. Data\Data 1.0\Geologi\Peta Geologi Daerah Penelitian\Polygon\Raster Peta Daerah Peneltian.tif"

In [ ]:
# Reading and normalizing input data
dsFeatures, arrFeatures = raster.read(featureFile, bands='all')
arrFeatures = arrFeatures.astype(float)

In [ ]:
for i in range(arrFeatures.shape[0]):
    bandMin = arrFeatures[i][:][:].min()
    bandMax = arrFeatures[i][:][:].max()
    bandRange = bandMax-bandMin
    for j in range(arrFeatures.shape[1]):
        for k in range(arrFeatures.shape[2]):
            arrFeatures[i][j][k] = (arrFeatures[i][j][k]-bandMin)/bandRange

In [ ]:
from copy import copy

In [ ]:
# Creating chips using pyrsgis
features = imageChipsFromArray(arrFeatures, x_size=7, y_size=7)

In [ ]:
# Reading and reshaping the label file
dsLabels, arrLabels = raster.read(labelFile)
arrLabels[arrLabels==255] = 0
arrLabels = arrLabels.flatten()

In [ ]:
# Separating and balancing the classes
features = features[arrLabels!=0]
labels = arrLabels[arrLabels!=0]

# Defining the function to split features and labels
def train_test_split(features, labels, trainProp=0.70):
    dataSize = features.shape[0]
    sliceIndex = int(dataSize*trainProp)
    randIndex = np.arange(dataSize)
    random.shuffle(randIndex)
    train_x = features[[randIndex[:sliceIndex]], :, :, :][0]
    test_x = features[[randIndex[sliceIndex:]], :, :, :][0]
    train_y = labels[randIndex[:sliceIndex]]
    test_y = labels[randIndex[sliceIndex:]]
    return(train_x, train_y, test_x, test_y)

In [ ]:
# Calling the function to split the data
train_x, train_y, test_x, test_y = train_test_split(features, labels)

In [ ]:
# Creating the model
# !pip install tensorflow
import tensorflow as tf

In [ ]:

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32, kernel_size=1, padding='valid', activation='relu', input_shape=(train_x.shape[1], train_x.shape[2], train_x.shape[3])))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Conv2D(48, kernel_size=1, padding='valid', activation='relu'))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

print(model.summary())


In [ ]:
# Running the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(train_x, train_y, epochs=20)

In [ ]:
# Loading and normalizing a new multispectral image
dsPre, featuresPre = raster.read(r"D:\KULIAH\Makan Bang\Argopuro\10. ML\Code\CNN\Dataset\Virtual Raster.tif")
featuresPre = featuresPre.astype(float)

for i in range(featuresPre.shape[0]):
    bandMinPre = featuresPre[i][:][:].min()
    bandMaxPre = featuresPre[i][:][:].max()
    bandRangePre = bandMaxPre-bandMinPre
    for j in range(featuresPre.shape[1]):
        for k in range(featuresPre.shape[2]):
            featuresPre[i][j][k] = (featuresPre[i][j][k]-bandMinPre)/bandRangePre

In [ ]:
# Generating image chips from the array
new_features = imageChipsFromArray(featuresPre, x_size=7, y_size=7)

In [ ]:
# Predicting new data and exporting the probability raster
newPredicted = model.predict(new_features)

In [ ]:
prediction = np.reshape(newPredicted.argmax(axis=1), (dsPre.RasterYSize, dsPre.RasterXSize))

In [ ]:
outFile = r'D:\KULIAH\Makan Bang\Argopuro\11. Hasil\CNN\CNN.tif'
raster.export(prediction, dsPre, filename=outFile, dtype='float')

In [8]:
cnn_map = prediction
print(cnn_map.shape)

NameError: name 'prediction' is not defined

In [ ]:
# Total samples
plt.figure(figsize=(15, 15))
plt.imshow(cnn_map, cmap='rainbow')
plt.show()

# CM-CNN-Accuracy Assessment

In [ ]:
# Predicting for test data 
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
yTestPredicted = model.predict(test_x)
y_score = yTestPredicted[:, 1:yTestPredicted.shape[1]]

In [ ]:
# Predicting for test data 
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
yTestPredicted = model.predict(test_x)
y_score = yTestPredicted[:, 1:yTestPredicted.shape[1]]

In [ ]:
# Calculating and displaying error metrics
yTestPredicted = (yTestPredicted>0.5).astype(int)

cMatrix = confusion_matrix(test_y, yTestPredicted.argmax(axis=1))
pScore = precision_score(test_y, yTestPredicted.argmax(axis=1), average='micro')
rScore = recall_score(test_y, yTestPredicted.argmax(axis=1), average='micro')
fscore = f1_score(test_y, yTestPredicted.argmax(axis=1), average='micro')

print("Confusion matrix:\n", cMatrix)
print("\nP-Score: %.3f, R-Score: %.3f, F-Score: %.3f" % (pScore, rScore, fscore))


In [ ]:
# CM-CNN-Accuracy Assessment

# Predicting for test data 
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
yTestPredicted = model.predict(test_x)
y_score = yTestPredicted[:, 1:yTestPredicted.shape[1]]

# Predicting for test data 
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
yTestPredicted = model.predict(test_x)
y_score = yTestPredicted[:, 1:yTestPredicted.shape[1]]

# Calculating and displaying error metrics
yTestPredicted = (yTestPredicted>0.5).astype(int)

cMatrix = confusion_matrix(test_y, yTestPredicted.argmax(axis=1))
pScore = precision_score(test_y, yTestPredicted.argmax(axis=1), average='micro')
rScore = recall_score(test_y, yTestPredicted.argmax(axis=1), average='micro')
fscore = f1_score(test_y, yTestPredicted.argmax(axis=1), average='micro')

print("Confusion matrix:\n", cMatrix)
print("\nP-Score: %.3f, R-Score: %.3f, F-Score: %.3f" % (pScore, rScore, fscore))

# ROC - CNN - Accuracy Assessment

In [ ]:
# ROC curve
from scipy import interp
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn.metrics import roc_curve, auc

In [ ]:
yTestPredicted = model.predict(test_x)
y_score = yTestPredicted[:, 1:yTestPredicted.shape[1]]

In [ ]:
# Computing ROC parameters for each class
n_classes = 9
fpr = dict()
tpr = dict()
roc_auc = dict()

In [ ]:
from sklearn.preprocessing import label_binarize
test_y = label_binarize(test_y, classes=list(range(1, n_classes+1)))

for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(test_y[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

In [ ]:
# Computing micro-average ROC parameters
fpr['micro'], tpr['micro'], _ = roc_curve(test_y.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [ ]:
# Computing macro-average ROC parameters
# First aggregating all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

In [ ]:
# Then interpolating all ROC curves at the points obtained from the line above
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])


In [ ]:
# Finally averaging and computing AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

In [ ]:
# Plotting
plt.figure(1)
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.4f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.4f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()


In [ ]:
# Plotting
plt.figure(1)
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.4f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.4f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='ROC curve of class {0} (area = {1:0.4f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()

plt.figure(2)
plt.xlim(0, 0.2)
plt.ylim(0.8, 1)
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.4f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.4f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='ROC curve of class {0} (area = {1:0.4f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()
